In [38]:
import random
import cvxpy as cp
import numpy as np

In [39]:
i = 3  # number of cars
j = 5  # number of incidents

b = [random.randint(1, 3) for i in range(j)]

x = cp.Variable((i, j), boolean=True)
z = cp.Variable(j, boolean=True)

objective = cp.Maximize(cp.sum(b @ z))
constraints = [cp.sum(x) >= b @ z, cp.sum(x) <= 1]
prob = cp.Problem(objective, constraints)


In [40]:
prob.solve(solver="GLPK_MI")

1.0

In [41]:
x.value

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [42]:
z.value

array([0., 0., 1., 0., 0.])

In [43]:
b

[3, 3, 1, 2, 1]

In [44]:
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']
